<a href="https://colab.research.google.com/github/SandeeeeeeeeepDey/data-science-11-weeks-progg/blob/main/california_housing_sequential_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [ ]:
cali_house_data = fetch_california_housing()
cali_data = pd.DataFrame(cali_house_data['data'], columns = cali_house_data["feature_names"])
cali_target = pd.Series(cali_house_data['target'], name = cali_house_data["target_names"][0])

In [ ]:
train_size = round(0.8*len(cali_data))
test_size = round(0.1*len(cali_data))
valid_size = round(0.1*len(cali_data))

In [ ]:
X_train, y_train = cali_data[:train_size], cali_target[:train_size]
X_valid, y_valid = cali_data[train_size : (train_size + valid_size)], cali_target[train_size : (train_size + valid_size)]
X_test, y_test = cali_data[(train_size + valid_size):], cali_target[(train_size + valid_size):]

In [ ]:
means_train = X_train.values.mean(axis = 0)
means_valid = X_valid.values.mean(axis = 0)
means_test = X_test.values.mean(axis = 0)

In [ ]:
stds_train = X_train.values.std(axis = 0)
stds_valid = X_valid.values.std(axis = 0)
stds_test = X_test.values.std(axis = 0)

In [ ]:
import torch

class CaliforniaHousing(Dataset):
  def __init__(self, data, target, means, stds):
    self.data = torch.tensor(data, dtype=torch.float32)
    self.target = torch.tensor(target).float()
    self.means = torch.tensor(means).float()
    self.stds = torch.tensor(stds).float()
    self.data = (self.data - self.means) / self.stds
    # self.data = (data.float - self.means) / self.stds

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    sample = {'data': self.data[idx], 'target': self.target[idx]}


    return sample

In [ ]:
ch_train = CaliforniaHousing(X_train.values, y_train.values, means_train, stds_train) # .values for turning df to np arrays, that the model understands
ch_valid = CaliforniaHousing(X_valid.values, y_valid.values, means_valid, stds_train)
ch_test = CaliforniaHousing(X_test.values, y_test.values, means_test, stds_test)

In [ ]:
len(ch_train)

16512

In [ ]:
ch_train[0]

{'data': tensor([ 2.4176,  0.9166,  0.5983, -0.1517, -0.9612, -0.0881,  1.2584, -1.6117]),
 'target': tensor(4.5260)}

In [ ]:
train_dl = DataLoader(ch_train, shuffle = True, batch_size = 32)
valid_dl = DataLoader(ch_train, shuffle = True, batch_size = 32)
test_dl = DataLoader(ch_test, shuffle = True, batch_size = 32)

In [ ]:
batch = next(iter(train_dl))
batch

{'data': tensor([[-1.2313e-01, -9.9876e-02,  3.5203e-02, -1.3724e-01,  1.9725e+00,
           1.6956e-01, -1.2193e+00,  1.0876e+00],
         [-9.7012e-01,  6.8205e-01, -5.8943e-01,  6.1705e-02,  8.3908e-02,
          -1.6384e-02, -6.9696e-01,  4.8949e-01],
         [ 3.4363e+00,  1.7767e+00,  8.5146e-01, -2.5694e-01, -3.2794e-01,
          -2.0273e-02, -5.4839e-01,  5.2589e-01],
         [ 3.7378e-01, -4.9084e-01, -2.1033e-01, -7.4541e-02, -4.5287e-01,
          -7.3967e-02, -6.7779e-01,  6.5592e-01],
         [-6.8458e-01, -9.9876e-02, -3.9490e-01, -5.2445e-02, -7.1181e-02,
           9.4365e-02, -5.8194e-01,  6.0391e-01],
         [ 4.6177e+00,  5.6509e-02,  1.0813e+00, -3.4100e-02, -7.1652e-01,
          -1.5400e-03, -1.1475e+00,  9.7837e-01],
         [-5.8274e-01,  1.3858e+00, -4.0046e-01,  8.9385e-02, -4.2099e-01,
          -8.0853e-02, -5.3880e-01,  4.7388e-01],
         [ 2.3015e-01,  1.0730e+00,  8.1153e-02, -1.2773e-01, -5.4765e-01,
          -2.1936e-02, -5.4360e-01,  5.519

In [ ]:
batch['data'].shape, batch['target'].shape

(torch.Size([32, 8]), torch.Size([32]))

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [ ]:
class CalNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense_relu_stack = nn.Sequential(
        nn.Linear(8,50),
        nn.ReLU(),
        nn.Linear(50,50),
        nn.ReLU(),
        nn.Linear(50,50),
        nn.ReLU(),
        nn.Linear(50, 1)
    )

  def forward(self, x):
    logits = self.dense_relu_stack(x)
    return logits

cali_model = CalNet().to(device)
cali_model

CalNet(
  (dense_relu_stack): Sequential(
    (0): Linear(in_features=8, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=1, bias=True)
  )
)

best practice

In [ ]:
# Sanity check:
output = cali_model(batch['data'])
output.shape

torch.Size([32, 1])

In [ ]:
loss_fn = nn.MSELoss()
# loss_fn = nn.
optimizer = torch.optim.Adam(cali_model.parameters(),lr = 1e-30)

In [ ]:
def train(model, dataloader, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch_idx, batch in enumerate(dataloader):
    X,y = batch['data'].to(device), batch['target'].to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch_idx % 100 == 0:
      loss, current = loss.item(), (batch_idx+1)*len(X)

      print(f"loss:{loss:>8f}, current:{current:>5d}, size:{size:>5d}")

In [ ]:
def validate(model, dataloader, loss_fn):
  size = len(dataloader.dataset)
  num_batch = len(dataloader)

  model.eval()

  loss_val, correct = 0,0
  with torch.no_grad():
    for batch in dataloader:
      X,y = batch["data"].to(device), batch["target"].to(device)
      pred = model(X)

      loss_val += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  loss_val /=  num_batch
  correct /= size

  print(f"acc:{100*correct:>0.1f}, loss:{loss_val:>8f}")

In [ ]:
def test(model, dataloader, loss_fn):
  size = len(dataloader.dataset)
  num_batch = len(dataloader)

  model.eval()
  loss_test, correct = 0,0

  with torch.no_grad():
    for batch in dataloader:
      X, y = batch["data"].to(device), batch["target"].to(device)

      pred = model(X)
      loss_test += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float). sum().item()
  loss_test /= num_batch
  correct /= size

  print(f"acc:{100*correct:>0.1f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

In [ ]:
epoch = 200

for i in range(epoch):
  print(f"Epoch:{i+1}")
  train(cali_model, train_dl, loss_fn, optimizer)
  validate(cali_model, valid_dl, loss_fn)

test(cali_model, test_dl, loss_fn)

Epoch:1
loss:9.459753, current:   32, size:16512


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss:5.213992, current: 3232, size:16512
loss:4.083831, current: 6432, size:16512
loss:8.643308, current: 9632, size:16512
loss:6.915877, current:12832, size:16512
loss:5.061926, current:16032, size:16512
acc:0.0, loss:5.903144
Epoch:2
loss:5.289101, current:   32, size:16512
loss:4.498654, current: 3232, size:16512
loss:5.913346, current: 6432, size:16512
loss:6.230417, current: 9632, size:16512
loss:6.245237, current:12832, size:16512
loss:7.537599, current:16032, size:16512
acc:0.0, loss:5.903219
Epoch:3
loss:5.316666, current:   32, size:16512
loss:5.236272, current: 3232, size:16512
loss:5.544516, current: 6432, size:16512
loss:6.645749, current: 9632, size:16512
loss:6.254523, current:12832, size:16512
loss:5.716540, current:16032, size:16512
acc:0.0, loss:5.903179
Epoch:4
loss:4.734822, current:   32, size:16512
loss:5.247935, current: 3232, size:16512
loss:8.590720, current: 6432, size:16512
loss:4.403290, current: 9632, size:16512
loss:4.606834, current:12832, size:16512
loss:

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
for batch, b in enumerate(train_dl):
  print(len(b["data"]), len(b["target"]))
  break

32 32


In [ ]:
from torch import nn

m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)

In [ ]:
input.shape

torch.Size([20, 16, 50])

In [ ]:
output.shape

torch.Size([20, 33, 24])

In [ ]:
output

tensor([[[-0.3243, -0.4492,  0.2652,  ...,  0.6540, -0.1057,  0.3143],
         [ 0.3161, -0.3348,  0.3848,  ...,  0.5359, -0.3953,  0.1528],
         [-0.4588, -0.0477,  0.4325,  ..., -0.8779, -0.3983, -0.3194],
         ...,
         [-0.5420, -0.0729,  1.1130,  ...,  0.8177,  0.8525,  0.7283],
         [ 0.3326, -0.2390,  0.5997,  ..., -0.2016, -0.2495, -1.2769],
         [ 0.2560,  0.1725,  0.3535,  ..., -0.4366, -0.7018, -0.0804]],

        [[ 0.8952, -0.3429, -0.0986,  ...,  1.1227,  0.9021, -0.3828],
         [ 0.3462,  0.7938, -0.2258,  ..., -1.0605, -0.5607,  0.3958],
         [-0.3360, -0.4385,  1.0437,  ..., -0.0650,  0.7328,  0.5957],
         ...,
         [-0.2901, -0.0522,  0.9529,  ...,  0.3130, -0.6872, -0.1049],
         [ 0.6848,  0.6586,  1.0826,  ...,  0.4932, -0.2576, -0.5638],
         [ 0.7025,  0.9691, -1.0630,  ..., -0.3514,  0.3527, -0.1663]],

        [[ 0.0202, -0.2409,  0.9111,  ..., -0.5531,  0.8125,  0.4361],
         [ 0.2961, -1.5905, -0.2433,  ...,  0